In [51]:
from credentials import *
import numpy as np
import pandas as pd
#import urllib
from urllib.request import urlopen
import json
import time

In [43]:
def get_friend_IDs(steamid, APIkey):
    getList_URL = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={APIkey}&steamid={steamid}&relationship=friend"
    file = urlopen(getList_URL)
    content = file.read().decode('utf-8')
    friendlist = json.loads(content)
    friendlist = pd.DataFrame(friendlist["friendslist"]["friends"])
    return friendlist["steamid"]

def get_friend_names(friend_IDs, APIkey):
    keys=["steamid", "personaname"]
    out=[]
    for id in friend_IDs:
        getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={APIkey}&steamids={id}"
        file = urlopen(getFriend_URL)
        content = file.read().decode('utf-8')
        friendstatus = json.loads(content)
        try:
            friendstatus = friendstatus["response"]["players"][0]
            out.append([friendstatus.get(key) for key in keys])
        except:
            continue
    return pd.DataFrame(out, columns=["steamid", "steamname"]).sort_values("steamname")

def fill_empty_ids(favs, friend_names):
    favs = pd.merge(favs, friend_names, on="steamname", suffixes=("","_2"))
    favs["steamid"]= favs["steamid_2"]
    favs.drop("steamid_2", axis=1, inplace=True)
    favs.to_csv("my_favorites.csv", sep=";", index=False)
    return favs

def check_fav_status(favs, APIkey):
    notify={}
    keys=["personaname", "steamid", "gameextrainfo"]
    for n in range(favs.shape[0]):
        id= favs.iloc[n]["steamid"]
        games= [x.strip() for x in favs.iloc[n]["games"].split(",")]
        getFriend_URL = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={APIkey}&steamids={id}"
        file = urlopen(getFriend_URL)
        content = file.read().decode('utf-8')
        friendstatus = json.loads(content)
        try:
            friendstatus = friendstatus["response"]["players"][0]
            name = friendstatus["personaname"]
            try:
                game = friendstatus["gameextrainfo"]
                if game in games:
                    notify[name] = game
            except: 
                notify[name] = ""
        except:
            continue
    return notify


def check_for_change(hits, notify):
    send = False
    for entry in hits.keys():
        if notify[entry] != hits[entry]:
            notify[entry] = hits[entry]
            if notify[entry] != "":
                send = True
    return notify, send

In [44]:
# use credentials to get steamID of entire friendlist
my_friend_IDs = get_friend_IDs(my_SteamID, my_APIkey)

# look up the steam names for all IDs in friendlist
my_friend_names = get_friend_names(my_friend_IDs, my_APIkey)

#load table of favorites
my_favs = pd.read_csv("my_favorites.csv", delimiter=";")

# fill empty IDs in favorites
my_favs = fill_empty_ids(my_favs, my_friend_names)

# initialize empty dict for notifications
my_notify = {}
for i in range(my_favs.shape[0]):
    my_notify[my_favs.iloc[i]["steamname"]]=""
my_notify

{'Pail': '', 'Smallboy_Slin': '', '[WC] Scott Sterling': '', 'chrisse286': ''}

In [55]:
getList_URL = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={my_APIkey}&steamid={my_SteamID}&relationship=friend"
file = urlopen(getList_URL)
content = file.read().decode('utf-8')
friendlist = json.loads(content)
friendlist["friendslist"]["friends"]

[{'steamid': '76561197960487372',
  'relationship': 'friend',
  'friend_since': 1360424195},
 {'steamid': '76561197962583100',
  'relationship': 'friend',
  'friend_since': 1397381161},
 {'steamid': '76561197970586960',
  'relationship': 'friend',
  'friend_since': 1612381871},
 {'steamid': '76561197977569434',
  'relationship': 'friend',
  'friend_since': 1345493291},
 {'steamid': '76561197979479074',
  'relationship': 'friend',
  'friend_since': 1348602855},
 {'steamid': '76561197979975796',
  'relationship': 'friend',
  'friend_since': 1353522397},
 {'steamid': '76561197981033122',
  'relationship': 'friend',
  'friend_since': 1348424175},
 {'steamid': '76561197982396704',
  'relationship': 'friend',
  'friend_since': 1352058037},
 {'steamid': '76561197985222289',
  'relationship': 'friend',
  'friend_since': 1376939017},
 {'steamid': '76561197989292226',
  'relationship': 'friend',
  'friend_since': 1354725719},
 {'steamid': '76561197990862686',
  'relationship': 'friend',
  'frien

In [4]:
#check repeatedly for changes
x=0
while x<10:
    my_hits = check_fav_status(my_favs, my_APIkey)
    my_notify, my_send = check_for_change(my_hits, my_notify)
    if my_send == True:
        for i in my_notify.keys():
            if my_notify[i] != "":
                print(f"{i} is playing {my_notify[i]}")
    else:
        print(".")
    time.sleep(10)
    x +=1

[WC] Scott Sterling is playing Sands of Salzaar
.


KeyboardInterrupt: 

In [23]:
from time import time, sleep, localtime, strftime


In [17]:
i = 0
while i < 10:
    start_time = time()
    print(start_time)
    #work work
    sleep(3)
    sleep(start_time + 10 - time())
    i += 1

1674047558.3622632
1674047568.3625503


KeyboardInterrupt: 

In [27]:
import requests

In [29]:
message = "testes"
url = f"https://api.telegram.org/bot{my_bot_token}/sendMessage?chat_id={my_telegram_id}&text={message}"
requests.get(url).json()

{'ok': True,
 'result': {'message_id': 17,
  'from': {'id': 5924291011,
   'is_bot': True,
   'first_name': 'SteamFavs-Notifier',
   'username': 'SteamFavs_Bot'},
  'chat': {'id': 108811623,
   'first_name': 'Valle',
   'last_name': 'So Yeah!',
   'username': 'UndValleSoYeah',
   'type': 'private'},
  'date': 1674048550,
  'text': 'testes'}}

In [40]:
int(37.00//3600)


0

In [36]:
(3662%3600)//60

1

In [37]:
(3662%60)

2

SyntaxError: unterminated string literal (detected at line 1) (2257429798.py, line 1)

In [58]:
test = json.load("my_favorites.csv")

AttributeError: 'str' object has no attribute 'read'

In [67]:
import csv
data_dict = {}
with open("my_favorites2.csv", encoding = 'utf-8') as csv_file_handler:
        csv_reader = csv.DictReader(csv_file_handler)
 
        #convert each row into a dictionary
        #and add the converted data to the data_variable
 
        for rows in csv_reader:
            print(rows)
            #assuming a column named 'No'
            #to be the primary key
            key = rows['friend']
            data_dict[key] = rows

{'friend': 'Schniin', 'steamname': 'Pail', 'steamid': '76561198059254917', 'games': 'Warhammer: Vermintide 2'}
{'friend': 'Nils', 'steamname': 'Smallboy_Slin', 'steamid': '76561197981033122', 'games': 'Deep Rock Galactic'}
{'friend': 'Grady', 'steamname': '[WC] Scott Sterling', 'steamid': '76561198026715831', 'games': 'Sands of Salzaar'}
{'friend': 'Chrisse', 'steamname': 'chrisse286', 'steamid': '76561198068931354', 'games': 'Dota 2'}


In [68]:
data_dict

{'Schniin': {'friend': 'Schniin',
  'steamname': 'Pail',
  'steamid': '76561198059254917',
  'games': 'Warhammer: Vermintide 2'},
 'Nils': {'friend': 'Nils',
  'steamname': 'Smallboy_Slin',
  'steamid': '76561197981033122',
  'games': 'Deep Rock Galactic'},
 'Grady': {'friend': 'Grady',
  'steamname': '[WC] Scott Sterling',
  'steamid': '76561198026715831',
  'games': 'Sands of Salzaar'},
 'Chrisse': {'friend': 'Chrisse',
  'steamname': 'chrisse286',
  'steamid': '76561198068931354',
  'games': 'Dota 2'}}

In [78]:
test= pd.DataFrame(

,friend,steamname,steamid,games
0,Schniin,Pail,76561198059254917,Warhammer: Vermintide 2
1,Nils,Smallboy_Slin,76561197981033122,Deep Rock Galactic
2,Grady,[WC] Scott Sterling,76561198026715831,Sands of Salzaar
3,Chrisse,chrisse286,76561198068931354,Dota 2


In [82]:
pd.read_csv("my_favorites.csv", delimiter=";").values.tolist()


[['Schniin', 'Pail', 76561198059254917, 'Warhammer: Vermintide 2, Test'],
 ['Nils', 'Smallboy_Slin', 76561197981033122, 'Deep Rock Galactic, SCUM'],
 ['Grady', '[WC] Scott Sterling', 76561198026715831, 'Sands of Salzaar'],
 ['Chrisse', 'chrisse286', 76561198068931354, 'Dota 2']]

In [86]:
test = 

In [87]:
pd.read_csv("my_favorites.csv", delimiter=";") == test

,friend,steamname,steamid,games
0,True,True,True,True
1,True,True,True,True
2,True,True,True,True
3,True,True,True,True
